# SCHOLAR - Scientific Comprehensive Holistic Objective Learning And Research
## Full Academic Research Workflow Platform

**Complete Research Pipeline:**
1. **SURVEY** - Systematic Literature Review & Gap Analysis
2. **HYPOTHESIS** - Research Question & Hypothesis Generation
3. **METHOD** - Methodology Design & Experimental Planning
4. **ANALYZE** - Statistical Analysis & Data Interpretation
5. **WRITE** - Academic Paper Drafting & Structure
6. **REVIEW** - Peer Review Simulation & Critique
7. **CITE** - Citation Management & Bibliography

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from datetime import datetime
from xhtml2pdf import pisa
import os, json, hashlib

load_dotenv()
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0.4)

class Config:
    DATA = 'scholar_data'
    REPORTS = f'{DATA}/papers'
    REVIEWS = f'{DATA}/reviews'
    PROJECTS = f'{DATA}/projects'

for d in [Config.DATA, Config.REPORTS, Config.REVIEWS, Config.PROJECTS]: os.makedirs(d, exist_ok=True)

CSS = '''<style>
@page{size:A4;margin:2.5cm}body{font-family:"Times New Roman",Georgia,serif;font-size:11pt;line-height:1.8;color:#1a1a1a;text-align:justify}
.header{text-align:center;margin-bottom:30px;border-bottom:2px solid #1a237e;padding-bottom:20px}
.header h1{font-size:16pt;margin:0 0 10px 0;color:#1a237e}.header .meta{font-size:10pt;color:#666}
h2{font-size:13pt;color:#1a237e;margin-top:25px;border-bottom:1px solid #e0e0e0;padding-bottom:5px}
h3{font-size:11pt;color:#37474f;margin-top:15px}
.abstract{background:#f5f5f5;padding:15px;border-left:4px solid #1a237e;margin:20px 0;font-style:italic}
.keyword{display:inline-block;background:#e3f2fd;padding:3px 10px;margin:2px;border-radius:15px;font-size:9pt}
.finding{background:#e8f5e9;padding:12px;margin:10px 0;border-radius:4px;border-left:4px solid #4caf50}
.gap{background:#fff3e0;padding:12px;margin:10px 0;border-radius:4px;border-left:4px solid #ff9800}
.critical{background:#ffebee;padding:12px;margin:10px 0;border-radius:4px;border-left:4px solid #f44336}
.info{background:#e3f2fd;padding:12px;margin:10px 0;border-radius:4px;border-left:4px solid #2196f3}
table{width:100%;border-collapse:collapse;margin:15px 0;font-size:10pt}
th{background:#1a237e;color:white;padding:10px;text-align:left}td{padding:10px;border:1px solid #e0e0e0}
tr:nth-child(even){background:#fafafa}
.ref{font-size:9pt;color:#666;margin:3px 0;padding-left:20px;text-indent:-20px}
.score{text-align:center;font-size:28pt;font-weight:700;color:#1a237e;padding:15px}
.footer{margin-top:30px;padding-top:15px;border-top:1px solid #ccc;font-size:8pt;color:#666;text-align:center}
</style>'''

class Project:
    @staticmethod
    def save(name, data):
        with open(f'{Config.PROJECTS}/{name}.json', 'w') as f: json.dump(data, f, indent=2, default=str)
    @staticmethod
    def load(name):
        path = f'{Config.PROJECTS}/{name}.json'
        return json.load(open(path)) if os.path.exists(path) else None
    @staticmethod
    def list_all():
        return [f.replace('.json', '') for f in os.listdir(Config.PROJECTS) if f.endswith('.json')]

def pdf(html, folder, name):
    ts = datetime.now()
    footer = f'<div class="footer">Generated by SCHOLAR | {ts.strftime("%Y-%m-%d %H:%M")} | ID: {hashlib.md5(html.encode()).hexdigest()[:8]}</div>'
    path = f'{Config.DATA}/{folder}/{name}_{ts.strftime("%Y%m%d_%H%M%S")}.pdf'
    with open(path, 'wb') as f: pisa.CreatePDF(CSS + html + footer, dest=f)
    print(f'PDF: {path}')
    return path

print('SCHOLAR Core Initialized')

In [ ]:
# SURVEY MODULE - Systematic Literature Review

class Paper(BaseModel):
    title: str
    authors: str
    year: int
    venue: str
    key_findings: List[str]
    methodology: str
    limitations: List[str]
    relevance: int = Field(ge=1, le=10)

class Theme(BaseModel):
    name: str
    description: str
    key_papers: List[str]
    consensus: str
    debates: List[str]

class LiteratureReview(BaseModel):
    topic: str
    search_strategy: str
    inclusion_criteria: List[str]
    papers_analyzed: int
    date_range: str
    themes: List[Theme]
    methodological_trends: List[str]
    theoretical_frameworks: List[str]
    research_gaps: List[str]
    future_directions: List[str]
    key_papers: List[Paper]

class GapAnalysis(BaseModel):
    field: str
    current_state: str
    gaps: List[Dict[str, str]]
    opportunities: List[str]
    barriers: List[str]
    priority_ranking: List[str]

SURVEY_PROMPT = '''You are a senior academic researcher conducting systematic literature reviews.
Follow PRISMA guidelines. Be rigorous, comprehensive, and cite real research trends.
Identify genuine gaps and future directions based on current state of research.'''

class SURVEY:
    def review(self, topic: str, scope: str = 'comprehensive') -> LiteratureReview:
        prompt = ChatPromptTemplate.from_messages([('system', SURVEY_PROMPT), ('human', 'Systematic literature review on: {topic}\nScope: {scope}')])
        r = (prompt | model.with_structured_output(LiteratureReview)).invoke({'topic': topic, 'scope': scope})
        
        themes_html = ''.join([f'''<div class="info"><h3>{t.name}</h3><p>{t.description}</p>
        <p><strong>Key Papers:</strong> {', '.join(t.key_papers)}</p>
        <p><strong>Consensus:</strong> {t.consensus}</p>
        <p><strong>Debates:</strong> {''.join([f'<br>- {d}' for d in t.debates])}</p></div>''' for t in r.themes])
        
        papers_html = ''.join([f'''<tr><td>{p.title}</td><td>{p.authors}</td><td>{p.year}</td><td>{p.venue}</td><td>{p.relevance}/10</td></tr>''' for p in r.key_papers])
        
        html = f'''<div class="header"><h1>Systematic Literature Review</h1><div class="meta">{r.topic}</div></div>
        <div class="abstract"><strong>Search Strategy:</strong> {r.search_strategy}<br>
        <strong>Date Range:</strong> {r.date_range} | <strong>Papers Analyzed:</strong> {r.papers_analyzed}</div>
        <h2>Inclusion Criteria</h2><ul>{''.join([f'<li>{c}</li>' for c in r.inclusion_criteria])}</ul>
        <h2>Thematic Analysis</h2>{themes_html}
        <h2>Methodological Trends</h2><ul>{''.join([f'<li>{m}</li>' for m in r.methodological_trends])}</ul>
        <h2>Theoretical Frameworks</h2><ul>{''.join([f'<li>{t}</li>' for t in r.theoretical_frameworks])}</ul>
        <h2>Research Gaps</h2>{''.join([f'<div class="gap">{g}</div>' for g in r.research_gaps])}
        <h2>Future Directions</h2>{''.join([f'<div class="finding">{d}</div>' for d in r.future_directions])}
        <h2>Key Papers</h2><table><tr><th>Title</th><th>Authors</th><th>Year</th><th>Venue</th><th>Relevance</th></tr>{papers_html}</table>'''
        
        print(f'Review: {r.topic} | {r.papers_analyzed} papers | {len(r.themes)} themes | {len(r.research_gaps)} gaps')
        pdf(html, 'reviews', f'lit_review_{topic.replace(" ", "_")[:30]}')
        return r
    
    def gaps(self, field: str) -> GapAnalysis:
        prompt = ChatPromptTemplate.from_messages([('system', SURVEY_PROMPT), ('human', 'Research gap analysis for: {field}')])
        r = (prompt | model.with_structured_output(GapAnalysis)).invoke({'field': field})
        
        gaps_html = ''.join([f'''<div class="gap"><h3>{g.get('gap', 'Gap')}</h3>
        <p><strong>Type:</strong> {g.get('type', 'N/A')} | <strong>Priority:</strong> {g.get('priority', 'Medium')}</p>
        <p>{g.get('description', '')}</p></div>''' for g in r.gaps])
        
        html = f'''<div class="header"><h1>Research Gap Analysis</h1><div class="meta">{r.field}</div></div>
        <h2>Current State of Knowledge</h2><div class="info">{r.current_state}</div>
        <h2>Identified Gaps</h2>{gaps_html}
        <h2>Research Opportunities</h2>{''.join([f'<div class="finding">{o}</div>' for o in r.opportunities])}
        <h2>Barriers to Progress</h2>{''.join([f'<div class="critical">{b}</div>' for b in r.barriers])}
        <h2>Priority Ranking</h2><ol>{''.join([f'<li><strong>{p}</strong></li>' for p in r.priority_ranking])}</ol>'''
        
        print(f'Gaps: {r.field} | {len(r.gaps)} gaps | {len(r.opportunities)} opportunities')
        pdf(html, 'reviews', f'gap_analysis_{field.replace(" ", "_")[:30]}')
        return r

survey = SURVEY()
print('SURVEY Module Ready')

In [ ]:
# HYPOTHESIS MODULE - Research Questions & Hypotheses

class ResearchQuestion(BaseModel):
    question: str
    type: str = Field(description='Descriptive/Exploratory/Explanatory/Causal')
    variables: List[str]
    theoretical_basis: str
    novelty: str
    feasibility: int = Field(ge=1, le=10)
    impact: int = Field(ge=1, le=10)

class Hypothesis(BaseModel):
    statement: str
    null_hypothesis: str
    alternative_hypothesis: str
    independent_variable: str
    dependent_variable: str
    control_variables: List[str]
    predicted_direction: str
    theoretical_rationale: str
    testability: int = Field(ge=1, le=10)

class ResearchDesign(BaseModel):
    title: str
    problem_statement: str
    research_questions: List[ResearchQuestion]
    hypotheses: List[Hypothesis]
    theoretical_framework: str
    contribution: str
    scope: str
    limitations: List[str]

HYPOTHESIS_PROMPT = '''You are a research methodologist helping design rigorous research.
Generate testable hypotheses with clear operationalization.
Ensure theoretical grounding and practical feasibility.'''

class HYPOTHESIS:
    def design(self, topic: str, gap: str = '') -> ResearchDesign:
        prompt = ChatPromptTemplate.from_messages([('system', HYPOTHESIS_PROMPT), ('human', 'Design research for: {topic}\nAddressing gap: {gap}')])
        r = (prompt | model.with_structured_output(ResearchDesign)).invoke({'topic': topic, 'gap': gap})
        
        rqs = ''.join([f'''<div class="info"><h3>RQ{i+1}: {q.question}</h3>
        <p><span class="keyword">{q.type}</span> <span class="keyword">Feasibility: {q.feasibility}/10</span> <span class="keyword">Impact: {q.impact}/10</span></p>
        <p><strong>Variables:</strong> {', '.join(q.variables)}</p>
        <p><strong>Theoretical Basis:</strong> {q.theoretical_basis}</p>
        <p><strong>Novelty:</strong> {q.novelty}</p></div>''' for i, q in enumerate(r.research_questions)])
        
        hyps = ''.join([f'''<div class="finding"><h3>H{i+1}: {h.statement}</h3>
        <table><tr><td><strong>H₀:</strong></td><td>{h.null_hypothesis}</td></tr>
        <tr><td><strong>H₁:</strong></td><td>{h.alternative_hypothesis}</td></tr>
        <tr><td><strong>IV:</strong></td><td>{h.independent_variable}</td></tr>
        <tr><td><strong>DV:</strong></td><td>{h.dependent_variable}</td></tr>
        <tr><td><strong>Controls:</strong></td><td>{', '.join(h.control_variables)}</td></tr></table>
        <p><strong>Rationale:</strong> {h.theoretical_rationale}</p>
        <p><span class="keyword">Testability: {h.testability}/10</span></p></div>''' for i, h in enumerate(r.hypotheses)])
        
        html = f'''<div class="header"><h1>{r.title}</h1><div class="meta">Research Design Document</div></div>
        <h2>Problem Statement</h2><div class="abstract">{r.problem_statement}</div>
        <h2>Research Questions</h2>{rqs}
        <h2>Hypotheses</h2>{hyps}
        <h2>Theoretical Framework</h2><div class="info">{r.theoretical_framework}</div>
        <h2>Expected Contribution</h2><div class="finding">{r.contribution}</div>
        <h2>Scope</h2><p>{r.scope}</p>
        <h2>Limitations</h2><ul>{''.join([f'<li>{l}</li>' for l in r.limitations])}</ul>'''
        
        print(f'Design: {r.title} | {len(r.research_questions)} RQs | {len(r.hypotheses)} hypotheses')
        pdf(html, 'papers', f'research_design_{topic.replace(" ", "_")[:30]}')
        return r

hypothesis = HYPOTHESIS()
print('HYPOTHESIS Module Ready')

In [ ]:
# METHOD MODULE - Methodology Design

class SampleDesign(BaseModel):
    population: str
    sampling_method: str
    sample_size: int
    power_analysis: str
    inclusion_criteria: List[str]
    exclusion_criteria: List[str]

class Instrument(BaseModel):
    name: str
    type: str
    reliability: str
    validity: str
    source: str

class Procedure(BaseModel):
    phase: str
    description: str
    duration: str
    materials: List[str]

class Methodology(BaseModel):
    research_paradigm: str
    approach: str = Field(description='Quantitative/Qualitative/Mixed')
    design: str
    sample: SampleDesign
    instruments: List[Instrument]
    procedure: List[Procedure]
    data_collection: str
    analysis_plan: List[str]
    ethical_considerations: List[str]
    limitations: List[str]
    timeline: str

METHOD_PROMPT = '''You are a research methodologist designing rigorous studies.
Follow best practices for the chosen paradigm (quantitative, qualitative, mixed).
Ensure validity, reliability, and ethical compliance.'''

class METHOD:
    def design(self, rq: str, approach: str = 'quantitative') -> Methodology:
        prompt = ChatPromptTemplate.from_messages([('system', METHOD_PROMPT), ('human', 'Design {approach} methodology for RQ: {rq}')])
        r = (prompt | model.with_structured_output(Methodology)).invoke({'rq': rq, 'approach': approach})
        
        instruments = ''.join([f'<tr><td>{i.name}</td><td>{i.type}</td><td>{i.reliability}</td><td>{i.validity}</td><td>{i.source}</td></tr>' for i in r.instruments])
        procedures = ''.join([f'''<div class="info"><h3>{p.phase}</h3>
        <p>{p.description}</p>
        <p><strong>Duration:</strong> {p.duration} | <strong>Materials:</strong> {', '.join(p.materials)}</p></div>''' for p in r.procedure])
        
        html = f'''<div class="header"><h1>Research Methodology</h1><div class="meta">{r.approach} | {r.design}</div></div>
        <h2>Research Paradigm</h2><div class="info">{r.research_paradigm}</div>
        <h2>Sampling</h2>
        <table><tr><td>Population</td><td>{r.sample.population}</td></tr>
        <tr><td>Method</td><td>{r.sample.sampling_method}</td></tr>
        <tr><td>Sample Size</td><td>n = {r.sample.sample_size}</td></tr>
        <tr><td>Power Analysis</td><td>{r.sample.power_analysis}</td></tr></table>
        <p><strong>Inclusion:</strong> {', '.join(r.sample.inclusion_criteria)}</p>
        <p><strong>Exclusion:</strong> {', '.join(r.sample.exclusion_criteria)}</p>
        <h2>Instruments</h2><table><tr><th>Name</th><th>Type</th><th>Reliability</th><th>Validity</th><th>Source</th></tr>{instruments}</table>
        <h2>Procedure</h2>{procedures}
        <h2>Data Collection</h2><div class="info">{r.data_collection}</div>
        <h2>Analysis Plan</h2><ol>{''.join([f'<li>{a}</li>' for a in r.analysis_plan])}</ol>
        <h2>Ethical Considerations</h2><ul>{''.join([f'<li>{e}</li>' for e in r.ethical_considerations])}</ul>
        <h2>Timeline</h2><p>{r.timeline}</p>
        <h2>Limitations</h2><ul>{''.join([f'<li>{l}</li>' for l in r.limitations])}</ul>'''
        
        print(f'Methodology: {r.approach} {r.design} | n={r.sample.sample_size} | {len(r.instruments)} instruments')
        pdf(html, 'papers', 'methodology')
        return r

method = METHOD()
print('METHOD Module Ready')

In [ ]:
# ANALYZE MODULE - Statistical Analysis

class StatisticalTest(BaseModel):
    test: str
    purpose: str
    assumptions: List[str]
    interpretation: str
    effect_size: str
    code: str

class AnalysisPlan(BaseModel):
    research_question: str
    variables: Dict[str, str]
    preliminary_tests: List[str]
    main_analyses: List[StatisticalTest]
    post_hoc: List[str]
    reporting_guidelines: str
    visualization: List[str]

class ResultsInterpretation(BaseModel):
    findings: List[str]
    statistical_summary: str
    effect_sizes: List[str]
    practical_significance: str
    limitations: List[str]
    implications: List[str]

ANALYZE_PROMPT = '''You are a statistician and data analyst.
Recommend appropriate statistical tests based on research design.
Provide Python/R code, effect sizes, and proper interpretation guidelines.'''

class ANALYZE:
    def plan(self, rq: str, variables: str, design: str) -> AnalysisPlan:
        prompt = ChatPromptTemplate.from_messages([('system', ANALYZE_PROMPT), ('human', 'Analysis plan for:\nRQ: {rq}\nVariables: {vars}\nDesign: {design}')])
        r = (prompt | model.with_structured_output(AnalysisPlan)).invoke({'rq': rq, 'vars': variables, 'design': design})
        
        tests = ''.join([f'''<div class="info"><h3>{t.test}</h3>
        <p><strong>Purpose:</strong> {t.purpose}</p>
        <p><strong>Assumptions:</strong> {', '.join(t.assumptions)}</p>
        <p><strong>Effect Size:</strong> {t.effect_size}</p>
        <div style="background:#263238;color:#b2ff59;padding:10px;font-family:monospace;font-size:9pt;border-radius:4px">{t.code.replace('<', '&lt;')}</div>
        <p><strong>Interpretation:</strong> {t.interpretation}</p></div>''' for t in r.main_analyses])
        
        html = f'''<div class="header"><h1>Statistical Analysis Plan</h1></div>
        <h2>Research Question</h2><div class="abstract">{r.research_question}</div>
        <h2>Variables</h2><table>{''.join([f'<tr><td><strong>{k}</strong></td><td>{v}</td></tr>' for k, v in r.variables.items()])}</table>
        <h2>Preliminary Tests</h2><ul>{''.join([f'<li>{t}</li>' for t in r.preliminary_tests])}</ul>
        <h2>Main Analyses</h2>{tests}
        <h2>Post-hoc Tests</h2><ul>{''.join([f'<li>{p}</li>' for p in r.post_hoc])}</ul>
        <h2>Visualization</h2><ul>{''.join([f'<li>{v}</li>' for v in r.visualization])}</ul>
        <h2>Reporting Guidelines</h2><div class="finding">{r.reporting_guidelines}</div>'''
        
        print(f'Analysis Plan | {len(r.main_analyses)} tests | {len(r.visualization)} visualizations')
        pdf(html, 'papers', 'analysis_plan')
        return r
    
    def interpret(self, results: str, context: str) -> ResultsInterpretation:
        prompt = ChatPromptTemplate.from_messages([('system', ANALYZE_PROMPT), ('human', 'Interpret results:\n{results}\nContext: {context}')])
        r = (prompt | model.with_structured_output(ResultsInterpretation)).invoke({'results': results, 'context': context})
        
        html = f'''<div class="header"><h1>Results Interpretation</h1></div>
        <h2>Key Findings</h2>{''.join([f'<div class="finding">{f}</div>' for f in r.findings])}
        <h2>Statistical Summary</h2><div class="info">{r.statistical_summary}</div>
        <h2>Effect Sizes</h2><ul>{''.join([f'<li>{e}</li>' for e in r.effect_sizes])}</ul>
        <h2>Practical Significance</h2><p>{r.practical_significance}</p>
        <h2>Implications</h2><ul>{''.join([f'<li>{i}</li>' for i in r.implications])}</ul>
        <h2>Limitations</h2><ul>{''.join([f'<li>{l}</li>' for l in r.limitations])}</ul>'''
        
        print(f'Interpretation | {len(r.findings)} findings | {len(r.implications)} implications')
        pdf(html, 'papers', 'results_interpretation')
        return r

analyze = ANALYZE()
print('ANALYZE Module Ready')

In [ ]:
# WRITE MODULE - Academic Paper Drafting

class Abstract(BaseModel):
    background: str
    objective: str
    methods: str
    results: str
    conclusion: str
    keywords: List[str]
    word_count: int

class Section(BaseModel):
    title: str
    content: str
    subsections: List[Dict[str, str]]

class PaperDraft(BaseModel):
    title: str
    abstract: Abstract
    introduction: Section
    literature_review: Section
    methodology: Section
    results: Section
    discussion: Section
    conclusion: Section
    references: List[str]

WRITE_PROMPT = '''You are an academic writing expert.
Follow APA/IEEE style as appropriate. Write clearly, concisely, and rigorously.
Ensure logical flow and proper academic tone.'''

class WRITE:
    def abstract(self, title: str, summary: str) -> Abstract:
        prompt = ChatPromptTemplate.from_messages([('system', WRITE_PROMPT), ('human', 'Write structured abstract for:\nTitle: {title}\nSummary: {summary}')])
        r = (prompt | model.with_structured_output(Abstract)).invoke({'title': title, 'summary': summary})
        
        html = f'''<div class="header"><h1>{title}</h1></div>
        <div class="abstract"><strong>Background:</strong> {r.background}<br><br>
        <strong>Objective:</strong> {r.objective}<br><br>
        <strong>Methods:</strong> {r.methods}<br><br>
        <strong>Results:</strong> {r.results}<br><br>
        <strong>Conclusion:</strong> {r.conclusion}</div>
        <p><strong>Keywords:</strong> {''.join([f'<span class="keyword">{k}</span>' for k in r.keywords])}</p>
        <p><em>Word count: {r.word_count}</em></p>'''
        
        print(f'Abstract | {r.word_count} words | {len(r.keywords)} keywords')
        pdf(html, 'papers', 'abstract')
        return r
    
    def section(self, section_name: str, context: str, guidelines: str = '') -> Section:
        prompt = ChatPromptTemplate.from_messages([('system', WRITE_PROMPT), ('human', 'Write {section} section:\nContext: {context}\nGuidelines: {guidelines}')])
        r = (prompt | model.with_structured_output(Section)).invoke({'section': section_name, 'context': context, 'guidelines': guidelines})
        
        subs = ''.join([f'<h3>{s.get("title", "")}</h3><p>{s.get("content", "")}</p>' for s in r.subsections])
        html = f'''<div class="header"><h1>{r.title}</h1></div>
        <p>{r.content}</p>{subs}'''
        
        print(f'Section: {r.title} | {len(r.subsections)} subsections')
        pdf(html, 'papers', f'section_{section_name}')
        return r

write = WRITE()
print('WRITE Module Ready')

In [ ]:
# REVIEW MODULE - Peer Review Simulation

class ReviewCriterion(BaseModel):
    criterion: str
    score: int = Field(ge=1, le=10)
    strengths: List[str]
    weaknesses: List[str]
    suggestions: List[str]

class PeerReview(BaseModel):
    summary: str
    recommendation: str = Field(description='Accept/Minor Revisions/Major Revisions/Reject')
    criteria: List[ReviewCriterion]
    major_concerns: List[str]
    minor_concerns: List[str]
    questions_for_authors: List[str]
    overall_score: int = Field(ge=1, le=10)

REVIEW_PROMPT = '''You are a rigorous peer reviewer for top academic journals.
Evaluate manuscripts critically but constructively.
Focus on methodology, contribution, clarity, and validity.'''

class REVIEW:
    def review(self, manuscript: str, venue: str = 'top journal') -> PeerReview:
        prompt = ChatPromptTemplate.from_messages([('system', REVIEW_PROMPT), ('human', 'Peer review for {venue}:\n{manuscript}')])
        r = (prompt | model.with_structured_output(PeerReview)).invoke({'manuscript': manuscript[:5000], 'venue': venue})
        
        criteria_html = ''.join([f'''<div class="{'finding' if c.score >= 7 else 'gap' if c.score >= 5 else 'critical'}">
        <h3>{c.criterion}: {c.score}/10</h3>
        <p><strong>Strengths:</strong> {', '.join(c.strengths)}</p>
        <p><strong>Weaknesses:</strong> {', '.join(c.weaknesses)}</p>
        <p><strong>Suggestions:</strong> {', '.join(c.suggestions)}</p></div>''' for c in r.criteria])
        
        rec_color = {'Accept': 'finding', 'Minor Revisions': 'info', 'Major Revisions': 'gap', 'Reject': 'critical'}
        
        html = f'''<div class="header"><h1>Peer Review Report</h1><div class="meta">{venue}</div></div>
        <div class="score">{r.overall_score}/10</div>
        <div class="{rec_color.get(r.recommendation, 'info')}" style="text-align:center;font-size:14pt;font-weight:bold">
        Recommendation: {r.recommendation}</div>
        <h2>Summary</h2><div class="abstract">{r.summary}</div>
        <h2>Evaluation Criteria</h2>{criteria_html}
        <h2>Major Concerns</h2>{''.join([f'<div class="critical">{c}</div>' for c in r.major_concerns])}
        <h2>Minor Concerns</h2><ul>{''.join([f'<li>{c}</li>' for c in r.minor_concerns])}</ul>
        <h2>Questions for Authors</h2><ol>{''.join([f'<li>{q}</li>' for q in r.questions_for_authors])}</ol>'''
        
        print(f'Review | {r.recommendation} | Score: {r.overall_score}/10')
        pdf(html, 'reviews', 'peer_review')
        return r

review = REVIEW()
print('REVIEW Module Ready')

In [ ]:
class SCHOLAR:
    def __init__(self):
        self.survey = survey
        self.hypothesis = hypothesis
        self.method = method
        self.analyze = analyze
        self.write = write
        self.review = review
    
    def projects(self):
        ps = Project.list_all()
        print(f'\n{len(ps)} Projects: {", ".join(ps) if ps else "None"}')
    
    def help(self):
        print('''
SCHOLAR - Full Academic Research Platform
==========================================

SURVEY (Literature):
  scholar.survey.review(topic)       Systematic lit review
  scholar.survey.gaps(field)         Research gap analysis

HYPOTHESIS (Design):
  scholar.hypothesis.design(topic)   RQs & Hypotheses

METHOD (Methodology):
  scholar.method.design(rq)          Methodology design

ANALYZE (Statistics):
  scholar.analyze.plan(rq,vars,design) Analysis plan
  scholar.analyze.interpret(results)   Interpret results

WRITE (Drafting):
  scholar.write.abstract(title,sum)  Write abstract
  scholar.write.section(name,ctx)    Write section

REVIEW (Peer Review):
  scholar.review.review(manuscript)  Simulate peer review

UTILITIES:
  scholar.projects()                 List saved projects
''')

scholar = SCHOLAR()
print('\n' + '='*50)
print('SCHOLAR READY - Run scholar.help()')
print('='*50)

In [ ]:
scholar.help()

In [ ]:
scholar.survey.review('Deep Learning for Medical Image Analysis')

In [ ]:
scholar.hypothesis.design('AI-assisted diagnosis accuracy', 'Lack of explainability in deep learning models')

In [ ]:
scholar.method.design('Does explainable AI improve physician trust compared to black-box models?', 'quantitative')

In [ ]:
scholar.analyze.plan('Effect of explainable AI on diagnostic accuracy', 'IV: AI type (XAI vs black-box), DV: accuracy, trust, adoption', 'Randomized controlled experiment')

In [ ]:
scholar.write.abstract('Explainable AI in Medical Diagnosis: A Randomized Controlled Trial', 'Study comparing XAI vs black-box models for radiology diagnosis, measured physician trust, accuracy, and adoption rates')

In [ ]:
scholar.review.review('This paper presents a study on explainable AI in medical imaging. We conducted an RCT with 120 radiologists comparing XAI and traditional deep learning models. Results show 23% improvement in trust and 15% improvement in diagnostic accuracy with XAI. The study has limitations including sample from single institution.')